### JSON 파일 읽기 및 데이터 추출
- `os.walk()`로 지정된 폴더 내 모든 JSON 파일 순회
- 각 파일에서 에세이 아이디, 텍스트, 평가 점수 등 필요한 정보 추출
- 추출한 데이터는 리스트에 저장

### 데이터 프레임 생성
- 추출된 데이터를 `pandas.DataFrame`으로 변환
- 중복된 에세이 아이디 제거
- 결과를 CSV 파일로 저장

### 텍스트 전처리 및 분석
- 텍스트에서 숫자 및 특수문자 제거
- 불용어 제거 후 형태소 분석
- 문장 수, 형태소 수, 고유 형태소 수 계산

### 데이터 병합
- `df`와 `stats_df`를 `essay_id` 기준으로 병합
- 최종 데이터프레임 `df_merge` 생성

### 특성 선택 및 전처리
- 카테고리형 변수(essay_type, student_reading)에 대해 OneHotEncoder 적용
- 숫자형 변수(essay_len, sentence_count, unique_word_count)에 대해 MinMaxScaler 적용
- ColumnTransformer를 사용하여 두 가지 전처리 방식 병행
- 전처리된 데이터를 X_transformed에 저장

In [1]:
import pandas as pd
import konlpy
import matplotlib
import seaborn
import sklearn

print(f"pandas=={pd.__version__}")
print(f"konlpy=={konlpy.__version__}")
print(f"matplotlib=={matplotlib.__version__}")
print(f"seaborn=={seaborn.__version__}")
print(f"scikit-learn=={sklearn.__version__}")

pandas==1.2.4
konlpy==0.5.2
matplotlib==3.3.4
seaborn==0.11.1
scikit-learn==0.24.1


In [2]:
import os
import json
import pandas as pd
import re
from konlpy.tag import Okt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

# 1. 데이터 로딩 및 필터링
folder_path = r"C:\Users\LEEYEJI\Downloads\과제\라벨링데이터"  # 데이터가 저장된 폴더 경로

data = []

# os.walk를 사용하여 모든 하위 폴더와 파일을 순회
for root, dirs, files in os.walk(folder_path):
    for file_name in files:
        if file_name.endswith('.json'):
            file_path = os.path.join(root, file_name)
            
            with open(file_path, 'r', encoding='utf-8') as f:
                json_data = json.load(f)
            
            # "student_grade"가 "고등_2학년"인 경우에만 데이터 추출
            if json_data["student"]["student_grade"] == "고등_2학년":
                # 각 문단별 정보 추출
                for paragraph in json_data["paragraph"]:
                    data.append({
                        # info: 에세이 기본 정보
                        "essay_id": json_data["info"].get("essay_id"),  # 에세이 아이디
                        "essay_type": json_data["info"].get("essay_type"),  # 에세이 유형
                        "essay_main_subject": json_data["info"].get("essay_main_subject"),  # 에세이 주제
                        "essay_level": json_data["info"].get("essay_level"),  # 에세이 난이도
                        "essay_prompt": json_data["info"].get("essay_prompt"),  # 에세이 프롬프트
                        "essay_len": json_data["info"].get("essay_len"),  # 에세이 글자수
                        
                        # student: 에세이 작성자 정보
                        "student_grade_group": json_data["student"].get("student_grade_group"),  # 학생 학년군
                        "student_grade": json_data["student"].get("student_grade"),  # 학생 학년
                        "student_reading": json_data["student"].get("student_reading"),  # 학생이 일주일 간 읽은 책의 양
                        "student_educated": json_data["student"].get("student_educated"),  # 학생의 논술 사교육 유무
                        "date": json_data["student"].get("date"),  # 에세이 수집일
                        "location": json_data["student"].get("location"),  # 에세이 수집 장소
                        
                        # paragraph: 에세이 문단별 정보 목록
                        "paragraph_id": paragraph.get("paragraph_id"),  # 문단 아이디
                        "paragraph_txt": paragraph.get("paragraph_txt"),  # 문단 텍스트
                        "paragraph_len": paragraph.get("paragraph_len"),  # 문단 글자수
                        
                        # score: 에세이 점수 정보
                        "essay_scoreT": json_data["score"].get("essay_scoreT"),  # 3명의 평가자 에세이 총점 목록
                        "essay_scoreT_avg": json_data["score"].get("essay_scoreT_avg"),  # 3명의 평가자 에세이 총점 평균
                        
                        # essay_scoreT_detail: 항목별 세부 정보
                        "essay_scoreT_exp": json_data["score"]["essay_scoreT_detail"].get("essay_scoreT_exp"),  # 표현 점수 목록
                        "essay_scoreT_org": json_data["score"]["essay_scoreT_detail"].get("essay_scoreT_org"),  # 구성 점수 목록
                        "essay_scoreT_cont": json_data["score"]["essay_scoreT_detail"].get("essay_scoreT_cont"),  # 내용 점수 목록
                        
                        # paragraph_score: 문단별 점수 정보
                        "paragraph_score": json_data["score"].get("paragraph_score"),  # 문단별 점수 정보
                        
                        # rubric: 루브릭 정보
                        "rubric_essay_type": json_data["rubric"].get("essay_type"),  # 에세이 유형
                        "rubric_essay_main_subject": json_data["rubric"].get("essay_main_subject"),  # 에세이 주제
                        "rubric_essay_grade": json_data["rubric"].get("essay_grade"),  # 에세이 작성자 학년
                        
                        # expression_weight: 루브릭 표현 가중치 정보
                        "exp": json_data["rubric"]["expression_weight"].get("exp"),  # 대분류 표현 가중치
                        "exp_grammar": json_data["rubric"]["expression_weight"].get("exp_grammar"),  # 문법 가중치
                        "exp_vocab": json_data["rubric"]["expression_weight"].get("exp_vocab"),  # 단어 가중치
                        "exp_style": json_data["rubric"]["expression_weight"].get("exp_style"),  # 문장표현 가중치
                        
                        # organization_weight: 루브릭 구성 가중치 정보
                        "org": json_data["rubric"]["organization_weight"].get("org"),  # 대분류 구성 가중치
                        "org_essay": json_data["rubric"]["organization_weight"].get("org_essay"),  # 문단 간 가중치
                        "org_paragraph": json_data["rubric"]["organization_weight"].get("org_paragraph"),  # 문단 내 가중치
                        "org_coherence": json_data["rubric"]["organization_weight"].get("org_coherence"),  # 일관성 가중치
                        "org_quantity": json_data["rubric"]["organization_weight"].get("org_quantity"),  # 분량 가중치
                        
                        # content_weight: 루브릭 내용 가중치 정보
                        "con": json_data["rubric"]["content_weight"].get("con"),  # 대분류 내용 가중치
                        "con_clearance": json_data["rubric"]["content_weight"].get("con_clearance"),  # 주제 명료성 가중치
                        "con_novelty": json_data["rubric"]["content_weight"].get("con_novelty"),  # 참신성 가중치
                        "con_prompt": json_data["rubric"]["content_weight"].get("con_prompt"),  # 프롬프트 독해력 가중치
                        "con_description": json_data["rubric"]["content_weight"].get("con_description"),  # 서술력 가중치
                    })

df = pd.DataFrame(data)

# 중복된 행 제거 (essay_id를 기준으로)
df = df.drop_duplicates(subset='essay_id', keep='first')

# CSV로 저장
df.to_csv('essay.csv', index=False, encoding='utf-8')

# 데이터 확인
df = pd.read_csv('essay.csv', encoding='utf-8')
print(df.head())
print(df.info())

# 2. 텍스트 전처리 및 통계 분석

# 텍스트 정규화 함수
def normalize_text(text):
    # 1. 숫자와 특수기호 제거
    text = re.sub(r'[^가-힣\s]', '', text)
    
    # 2. 중복된 공백 제거
    text = re.sub(r'\s+', ' ', text)
    
    # 3. 공백 앞뒤 여백 제거
    text = text.strip()
    
    return text

# 형태소 분석기
okt = Okt()

korean_stopwords = [
    "의", "가", "이", "은", "는", "들", "을", "를", "에", "와", "과"
]

# 문단별로 분석 결과를 저장할 리스트 초기화
statistics = []

# 데이터프레임의 각 문단에 대해 통계 분석 수행
for index, row in df.iterrows():
    paragraph_txt = row['paragraph_txt']
    
    # '#@문장구분#' 제거한 텍스트 (cleaned_text용)
    cleaned_text = paragraph_txt.replace('#@문장구분#', '')
    
    # 문장 수: '#@문장구분#'을 기준으로 분리 (문장구분 횟수 = 구분자 개수)
    sentence_count = paragraph_txt.count('#@문장구분#')
    
    # 정규화된 텍스트
    normalized_text = normalize_text(cleaned_text)
    
    # 형태소 분석
    pos_tags = okt.pos(normalized_text)
    
    # 전체 형태소 수
    word_count = len(pos_tags)
    
    # 고유 형태소 수
    unique_word_count = len(set([word for word, pos in pos_tags]))
    
    # 불용어 제거
    filtered_words = [word for word, pos in pos_tags if word not in korean_stopwords]
    
    # 불용어 제거 후 텍스트
    cleaned_text = " ".join(filtered_words)
    
    statistics.append({
        'essay_id': row['essay_id'],
        'word_count': word_count,
        'sentence_count': sentence_count,
        'unique_word_count': unique_word_count,
        'cleaned_text': cleaned_text
    })

# 통계 데이터프레임 생성
stats_df = pd.DataFrame(statistics)

# 통합 데이터프레임 (원본 데이터프레임과 통계 데이터프레임 병합)
df_merge = pd.merge(df, stats_df, on=['essay_id'], how='left')

# 데이터 확인
print(df_merge.head())
print(df_merge.info())

# 카테고리형 변수 및 숫자형 변수 정의
categorical_features = ['essay_level', 'essay_type', 'student_reading', 'exp', 'exp_vocab', 'exp_style', 'org', 'org_coherence', 'con_clearance', 'con_prompt', 'con_description']
numeric_features = ['essay_len', 'paragraph_len', 'sentence_count', 'unique_word_count']

X = df_merge[categorical_features + numeric_features]

y = df_merge['essay_scoreT_avg']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ], 
)

X_transformed = preprocessor.fit_transform(X)
print(X_transformed)

      essay_id essay_type essay_main_subject  essay_level  \
0  ESSAY_34071        글짓기       우주에 대한 나의 생각            2   
1  ESSAY_35733        글짓기       우주에 대한 나의 생각            2   
2  ESSAY_35734        글짓기       우주에 대한 나의 생각            2   
3  ESSAY_35735        글짓기       우주에 대한 나의 생각            2   
4  ESSAY_35736        글짓기       우주에 대한 나의 생각            2   

                                        essay_prompt  essay_len  \
0   여러분은 '우주'라는 것에 대해 어떻게 생각하나요?\n\n 한 번도 생각해보지 않...        840   
1   여러분은 '우주'라는 것에 대해 어떻게 생각하나요?\n\n 한 번도 생각해보지 않...       1211   
2   여러분은 '우주'라는 것에 대해 어떻게 생각하나요?\n\n 한 번도 생각해보지 않...        662   
3   여러분은 '우주'라는 것에 대해 어떻게 생각하나요?\n\n 한 번도 생각해보지 않...       1019   
4   여러분은 '우주'라는 것에 대해 어떻게 생각하나요?\n\n 한 번도 생각해보지 않...        715   

  student_grade_group student_grade  student_reading  student_educated  ...  \
0                  고등        고등_2학년                2              True  ...   
1                  고등        고등_2학년                1              True  